In [11]:
import logging
import os
import sys
import pickle
import time


import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.autograd import Variable
from tqdm import tqdm


from sklearn.metrics import accuracy_score


# test = pd.read_csv(r"corpus\imdb\testData.tsv", header=0, delimiter="\t", quoting=3)


class SentimentNet(nn.Module):
    def __init__(self, embed_size, num_filter, filter_size, weight, labels, use_gpu, **kwargs):
        super(SentimentNet, self).__init__(**kwargs)

        self.use_gpu = use_gpu
        # 话说这个不会是我前面做的单词到词向量的映射表吧？？ 答：不用怀疑，正是
        self.embedding = nn.Embedding.from_pretrained(weight)
        # 防止嵌入层在训练中更新
        self.embedding.weight.requires_grad = False

        # 啥玩意儿这是？
        self.conv1d = nn.Conv1d(embed_size, num_filter, filter_size, padding=1)
        self.activate = F.relu
        self.decoder = nn.Linear(num_filter, labels)


    def forward(self, inputs):
        embeddings = self.embedding(inputs)

        convolution = self.activate(self.conv1d(embeddings.permute([0, 2, 1])))
        pooling = F.max_pool1d(convolution, kernel_size=convolution.shape[2])
        outputs = self.decoder(pooling.squeeze(dim=2))
        # print(outputs)
        return outputs



In [12]:
num_epochs = 10
embed_size = 628
num_filter = 32
filter_size = 3
bidirectional = True
batch_size = 256
labels = 2
lr = 0.8
device = torch.device('cuda:0')
use_gpu = True

model = torch.load('model_cnn.pkl')

pickle_file = 'depression_glove_42B_300d.pkl'
[train_features, train_labels, val_features, val_labels, test_features, weight, word_to_idx, idx_to_word, vocab] = pickle.load(open(pickle_file, 'rb'))

train_data = torch.concat([train_features, val_features], axis=0)
labels = torch.concat([train_labels, val_labels], axis=0)

print(train_data.shape)
val_set = torch.utils.data.TensorDataset(train_data, labels)
val_iter = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)

torch.Size([4860, 628])


In [13]:
labels.shape

torch.Size([4860])

In [14]:
features_layer = []
output_layer = []
def hook(module, input, output):
    features_layer.append(input)
    output_layer.append(output)
    return None


# 确定想要提取出的中间层名字
for (name, module) in model.named_modules():
    print(name)
# #  设置钩子
model.decoder.register_forward_hook(hook)
# a = torch.randn((3,1,28,28))
# net(a)
# print(features)



embedding
conv1d
decoder


In [15]:
import torch
import csv

# model = torch.nn.Sequential(*list(model.children())[:-1])


In [16]:
train_data.shape

torch.Size([4860, 628])

In [17]:
labels.shape

torch.Size([4860])

In [18]:
len(labels)

4860

In [19]:
# 设置模型为评估模式
model.eval()

# 4. 提取特征并写入CSV文件
with open('features.csv', 'w', newline='') as csvfile:
    csvfile_labels = open('labels.csv', 'w', newline='')
    writer = csv.writer(csvfile)
    writer_label = csv.writer(csvfile_labels)

    # # 写入表头
    # header = [ 'Features','Label']
    # writer.writerow(header)

    # 遍历数据集并提取特征
    for images, labels in val_iter:
        images = images.to('cuda:0')
        features = model(images)
        # print(features.shape)

        if features.shape[0] == 252:
            features = features_layer[0][0][:252,:]
            # print('到了')
        else:
            features = features_layer[0][0]
        # features = features_layer[0][0]
        # print(features.shape)

        features = features.tolist()
        labels = labels.tolist()
        for i in range(len(features)):
            writer.writerow(features[i])
            writer_label.writerow(str(labels[i]))




In [20]:
temp_train = pd.read_csv('features.csv',header=None)
temp_label = pd.read_csv('labels.csv',header=None)
print(temp_train.shape)
print(temp_label.shape)

(4860, 128)
(4860, 1)
